In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import eden
import Valium.sumsim as ss 

/home/ikea/.local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/ikea/miniconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ikea/miniconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarni

* sumsim.score() 
  * done

* build random search thing
  * first do the eval10 
  * check its speed

In [2]:

defaultarg={}
defaultarg['imp_thresh']=.3
defaultarg['imp_lin_start']=.1
defaultarg['maxsizediff']=10
defaultarg['acc_min_sim']=.5
defaultarg['n_samples']=3
defaultarg['n_steps']=50
defaultarg['quick_skip']=True
defaultarg['core_choice']=True
defaultarg['burnin']=10
defaultarg['mincipcount']=2 
defaultarg['mininterfacecount']=2 
                      
    
import graphlearn.abstract_graphs.RNA as rna
from graphlearn.estimator import Wrapper as estimatorwrapper

from graphlearn.graphlearn import Sampler as GLS
import random

NJOBS=1

def fit_sample(graphs, random_state=random.random(),arguments=defaultarg):
    '''
    graphs -> more graphs
    arguments are generated above Oo
    '''
    #print 'i am the sampler and i use this: '
    #print arguments
    graphs = list(graphs)
    estimator=estimatorwrapper( nu=.5, cv=2, n_jobs=NJOBS)
    sampler=rna.AbstractSampler(radius_list=[0,1],
                                thickness_list=[2], 
                                min_cip_count=arguments['mincipcount'], 
                                min_interface_count=arguments['mininterfacecount'], 
                                preprocessor=rna.PreProcessor(base_thickness_list=[1],
                                    ignore_inserts=True), 
                                postprocessor=rna.PostProcessor(),
                                estimator=estimator
                                #feasibility_checker=feasibility
                               )
    sampler.fit(graphs,grammar_n_jobs=NJOBS,grammar_batch_size=1)
    graphs = [ b for a ,b in graphs  ]
    graphs = sampler.sample(graphs,
                            n_samples=arguments['n_samples'],
                            batch_size=1,
                            n_steps=arguments['n_steps'],
                            n_jobs=NJOBS,
                            quick_skip_orig_cip=arguments['quick_skip'],
                            probabilistic_core_choice=arguments['core_choice'],
                            burnin=arguments['burnin'],
                            improving_threshold=arguments['imp_thresh'],
                            improving_linear_start=arguments['imp_lin_start'],
                            max_size_diff=arguments['maxsizediff'],
                            accept_min_similarity=arguments['acc_min_sim'],
                            select_cip_max_tries=30,
                            keep_duplicates=False,
                            include_seed=False,
                            backtrack=2,
                            monitor=False)
    result=[]
    for graphlist in graphs:
        result+=graphlist
    # note that this is a list [('',sequ),..]
    return result

In [3]:
%%time
import Valium.randset as rs

import curve
# das hier chillt in der curve rum
def get_data():
    return curve.get_seq_tups(fname='RF00005.fa',size=10,sizeb=50)

def run_and_score(argz):
    a,b= get_data()
    b=fit_sample(a,arguments=argz)
    a,b=ss.unpack(a,b)
    print "generated_seqs %d" % len(b)
    score = ss.score(a,b)
    return score
    

def meaning(argz,num=9):
    scores=[run_and_score(argz) for i in range(num)]
    scores.sort()
    #print scores
    return scores[num/2]
    
    
def zeloop():
    currenthigh=0
    while True:
        argz=rs.get_random_params()
        res=meaning(argz,num=1)
        print res
        if currenthigh < res:
            currenthigh=res
            print '\n'+str(argz)


zeloop()

#0.95 on single try: {'mininterfacecount': 2, 'burnin': 8, 'acc_min_sim': 0.35056045630513755, 'imp_lin_start': 0.094488063143386425, 'maxsizediff': 5, 'imp_thresh': 0.75498425474899822, 'mincipcount': 1, 'core_choice': True, 'n_samples': 4, 'n_steps': 91, 'quick_skip': True}

/home/ikea/.local/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


[[ 0.57605068  1.          0.03300492  0.15936184  0.        ]
 [ 0.57605068  1.          0.03300492  0.15936184  0.        ]
 [ 0.03615508  0.03300492  1.          0.          0.05555556]
 [ 0.12523577  0.15936184  0.          1.          0.        ]
 [ 0.          0.          0.05555556  0.          1.        ]]


TypeError: fit() got an unexpected keyword argument 'n_jobs'

In [ ]:
# ok so we need to do something about the random choosing of parameters

from sklearn.model_selection import ParameterSampler
from scipy.stats.distributions import uniform,randint
import numpy as np
import time
np.random.seed()

uniform(loc=4,scale=2) # default is something simething 


param_grid = {'a':randint(high=100,low=20), 'b':uniform(loc=4,scale=2) }


param_list = list(ParameterSampler(param_grid, n_iter=4))

#rounded_list = [dict((k, round(v, 6)) for (k, v) in d.items())
#                 for d in param_list]

print param_list

In [ ]:
import sklearn
print sklearn.__version__
print sklearn.__path__

In [ ]:
import Valium.randset as rs
rs.get_random_params()
#valium.randset.get_rangm_params()

In [ ]:
list(p)

In [ ]:

[-0.39702214422186499, -0.29200198556414503, -0.23544644219052113, -0.13913319572382932, -0.11955274813825978, -0.11006759906215913, -0.05867221251570709, 0.01196289322862687, 0.20323747543128967]
m5: -0.119552748138
0.203947871653 0.927714857333
0.139414680807 0.912990709065
0.10424604653 0.941974273617
0.287574385355 0.943667989137
0.223795121842 0.934594796208
0.205923304653 0.915279232387
0.107857802828 0.931537376288
0.189992903937 0.907981206043
0.216386939678 0.933742020397
[-0.52499008749907328, -0.50010616497501248, -0.35533198583878145, -0.14800959029429872, -0.11192337072090097, -0.091586013774874075, -0.068194261686684743, -0.039414308840014622, 0.20662955228438296]
m5: -0.111923370721
0.13650855788 0.908668140188
0.228852682033 0.93764202341
0.227710276282 0.941995166114
0.322031176057 0.924567909516
0.181538174062 0.939154784505
0.239671204923 0.93055542938
0.419303811476 0.917646144177
0.311784445087 0.932030003214
0.0667663145754 0.926375601651
[-0.65931034334969274, -0.36263390866736733, -0.21300208825780831, -0.031154060986744447, -0.022231295278711927, 0.028129390310982161, 0.31510777713386595, 0.36355679471031199, 0.75956910172750425]
m5: -0.0222312952787
0.182736163896 0.958142260911
0.268660094246 0.931641134499
0.241876309165 0.913429605546
0.15328634145 0.881047984133
0.320868355443 0.921035055245
0.28817706666 0.902789980017
0.22922674202 0.935997737627
0.189631218289 0.933685388867
0.248409415488 0.924973051808
[-0.26790261833226769, -0.22719760532558175, -0.17516051571284641, -0.019090769545726527, 0.054075631112182054, 0.06866461014537284, 0.14299924248711982, 0.24991828662297733, 0.36243836652717221]
m5: 0.0540756311122
0.255023333164 0.91641407612
0.287871690465 0.92326234863
0.259961339886 0.935552165107
0.431846144624 0.911242190043
0.206478653277 0.942047827712
0.106195180405 0.945608563674
0.193675538782 0.909334659738
0.287278638051 0.946230750258
0.336137402611 0.905824553927
[-0.52082784205479671, -0.13463250460820442, -0.11613321460216541, 0.10367925653823107, 0.10429319443732976, 0.20288380194740596, 0.22822441323201248, 0.43872505651701776, 0.81614238845271914]

In [ ]:
def avg(things):
    return sum(things)/float(len(things))
s=[-0.39702214422186499, -0.29200198556414503, -0.23544644219052113, -0.13913319572382932, -0.11955274813825978, -0.11006759906215913, -0.05867221251570709, 0.01196289322862687, 0.20323747543128967]
#m5: -0.119552748138
print avg(s)
s=[-0.52499008749907328, -0.50010616497501248, -0.35533198583878145, -0.14800959029429872, -0.11192337072090097, -0.091586013774874075, -0.068194261686684743, -0.039414308840014622, 0.20662955228438296]
#m5: -0.111923370721
print avg(s)
s=[-0.65931034334969274, -0.36263390866736733, -0.21300208825780831, -0.031154060986744447, -0.022231295278711927, 0.028129390310982161, 0.31510777713386595, 0.36355679471031199, 0.75956910172750425]
#m5: -0.0222312952787
print avg(s)
s=[-0.26790261833226769, -0.22719760532558175, -0.17516051571284641, -0.019090769545726527, 0.054075631112182054, 0.06866461014537284, 0.14299924248711982, 0.24991828662297733, 0.36243836652717221]
print avg(s)
#m5: 0.0540756311122
s=[-0.52082784205479671, -0.13463250460820442, -0.11613321460216541, 0.10367925653823107, 0.10429319443732976, 0.20288380194740596, 0.22822441323201248, 0.43872505651701776, 0.81614238845271914]
print avg(s)